In [1]:
%run -i '../../../utils/setup_environment.ipy'

import warnings
warnings.filterwarnings('ignore')
from scipy.stats import fisher_exact, ranksums, chi2, norm
from statsmodels.sandbox.stats.multicomp import multipletests
import matplotlib.gridspec as gridspec

data_path = '../../../data/'

Setup environment... done!


<span style="color:green">✅ Working on **mskimpact_env** conda environment.</span>

In [2]:
arm_level = pd.read_csv(data_path + 'impact-facets-tp53/new_data_facets/msk_impact_facets_annotated.arm_level.txt', sep='\t')
gene_level = pd.read_csv(data_path + 'impact-facets-tp53/new_data_facets/msk_impact_facets_annotated.gene_level_grepTP53.txt', sep='\t')

In [ ]:
# TO CHECK
# Inconsistencies gene_level - arm_level in tp53
from tqdm import tqdm,tqdm_notebook
tqdm_notebook().pandas()
def inconsistency(x):
    if arm_level[arm_level['sample'] == x['sample']][arm_level['arm'] == '17p']['cn_state'].shape[0]>0:
        if arm_level[arm_level['sample'] == x['sample']][arm_level['arm'] == '17p']['cn_state'].values[0] != x.cn_state:
            return True
        else: return False
    else: return False

    

gene_level_tp53 = gene_level[gene_level['gene'] == 'TP53']
gene_level_tp53['inconcistency'] = gene_level_tp53.progress_apply(inconsistency, axis=1)